## Actividad 1: Generación de ETL

El proceso de ETL (Extract, Transform an Load) es escencial en el uso de modelos de "machine learning", este proceso se encarga de la preparación de los datos para el proceso de entrenamiento de los modelos. El proceso de ETL es crítico para el éxito de cualquier proyecto de Machine Learning, ya que los modelos solo son tan buenos como los datos que se utilizan para entrenarlos. Por lo tanto, la calidad de los datos y la precisión del proceso de ETL son esenciales para la obtención de resultados precisos y confiables.

En la presente actividad prepararemos un dataset con informacion de los precios de alquiler de casas/apartamentos para la ciudad de Seatle según AirBNB, a partir de este dataset se podría entrenar un modelo capas de estimar el valor del alquiler de un inmueble y por ejemplo publicarlo en Airbnb

In [ ]:
# Dython es una libreria que nos ayuda a encontrar el grado de correlación que existen entre las variables, no viene instalado en las instancias de colab, pero se puede instalar facilmente, depende de nyumpy 1.23.0
!pip install numpy==1.23.0 
!pip install dython

## Imports 
Librerias que se necesitan para el desarrollo de la actividad

In [ ]:
import datetime

import numpy as np
import pandas as pd

from dython import nominal

#### Configurando pandas para visualización de 100 columnas

In [ ]:
pd.options.display.max_columns = 100

### Extracción de dataset sin preprocesamiento
El dataset es un archivo .csv y se encuantra alojado en el siguiente repositorio de GitHub: 
[repositorio](https://github.com/eyberthrojas/Actividad_1.git), pero lo podemos extraer directamente como un "dataframe" de pandas:

In [ ]:
path = 'https://raw.githubusercontent.com/eyberthrojas/Actividad_1/main/Airbnd_seatle_listings.csv'
df = pd.read_csv(path, on_bad_lines='skip')

In [ ]:
# Tamaño del dataset:
print(f'Cantidad de filas o registros = {df.shape[0]}')
print(f'Cantidad de columnas o variables = {df.shape[1]}')

In [ ]:
# Nombre de las variables(columnas) del dataframe
print('Nombre de las columnas: ')
df.columns

In [ ]:
# Primeros tres registros del dataframe
df.head(3)

La tabla anterior nos muestra las columnas numéricas del dataset, responda las preguntas  

## Ajuste de variables
Antes que nada, como vemos existen columnas como `host_response_rate` que tienen simbolos ('%', '$') que representan un tipo de cantidad numérica, pero en si, las variables deberian ser numéricas y así las debemos tratar. Con pandas se tratan de tipo str, por lo que se pueden adecuar para que sean numéricas

### Eliminando porcentaje

Como sabemos que la columna es de tipo str, podemos aplicar el metodo replace para columnas de tipo str. Con esto podemos eliminar el caracter "%", lo convertimos a float y por último se divide en 100 para que queden valores estandar entre 0 y 1


In [ ]:
df['host_response_rate'] = df['host_response_rate'].str.replace("%","").astype(float)/100
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace("%","").astype(float)/100


### Eliminando tipo moneda

Al igual que con el porcentaje "%", el signo de moneda tambien se podria eliminar para que las columnan que muestran precio queden de tipo numérico
Cómo sabemos que la columna es de tipo str, podemos aplicar el metodo replace para columnas de tipo str. Con esto podemos eliminar el caracter "%", lo convertimos a float y por último se divide en 100 para que queden valores estandar entre 0 y 1

In [ ]:
df['price'] = df['price'].str.replace(",","").str.replace("$","").astype(float)
df['security_deposit'] = df['security_deposit'].str.replace(",","").str.replace("$","").astype(float)
df['cleaning_fee'] = df['cleaning_fee'].str.replace(",","").str.replace("$","").astype(float)
df['extra_people'] = df['extra_people'].str.replace(",","").str.replace("$","").astype(float)

Por otro lado, existen columns que son de tipo bool pero esdtán tabuladas como 't' o 'f', también se puden pasar este tipo de variables a tipo bool de pandas. Cómo son varias realizamos un bucle con las columnas que queremos procesar:

In [ ]:
columns_to_bolean = [
    'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 
    'is_location_exact', 'has_availability', 'requires_license', 'instant_bookable', 
    'require_guest_profile_picture', 'require_guest_phone_verification'
]
for name in columns_to_bolean:
  df[name] = df[name].apply(lambda x: 1 if x=='t' else 0 if x=='f' else None)

Para finalizar, existen variables de fecha que nos podrian servir, pero debemos acondicionarla para que sea útil la información.

Por ejemplo la variables `host_since`, `first_review` y `last_review` se refieren a fechas, desde cuando es anfitrion, cuando fue el primer y el último review del inmueble. De pronto nos sirve la cantidad de dias que lleva como host, o por ejemplo hace cuantos dias fue el último review: 

In [ ]:
today = pd.to_datetime(datetime.date.today())

df['host_since'] = (today - pd.to_datetime(df['host_since'])).dt.days
df['first_review'] = (today - pd.to_datetime(df['first_review'])).dt.days
df['last_review'] = (today - pd.to_datetime(df['last_review'])).dt.days


In [ ]:
df.head()

### Metodo describe de pandas
El método `describe()` de pandas es una función que se utiliza para obtener estadísticas descriptivas de un DataFrame o una Serie de pandas. Esta función calcula y muestra varios estadísticos de resumen de los datos, como la media, la desviación estándar, los percentiles y los valores mínimos y máximos.

Algunos de los estadísticos que se muestran con el método `describe()` son:

- `count`: número de valores no nulos.\
- `mean`: promedio de los valores.\
- `std`: desviación estándar de los valores.\
- `min`: valor mínimo.\
- `25%`: percentil 25 (el valor por debajo del cual cae el 25% de los datos).\
- `50%`: percentil 50, también conocido como la mediana.\
- `75%`: percentil 75 (el valor por debajo del cual cae el 75% de los datos).\
- `max`: valor máximo.

Por defecto, `describe()` sólo muestra estadísticos para columnas numéricas, aunque se puede utilizar el parámetro include para incluir columnas de otros tipos de datos, como cadenas de caracteres.


In [ ]:
df.describe()

A partir de esta tabla, que describe las variables numéricas, responda las pregundas de 1-7

### Variables categóricas
Las variables categóricas nos sirven para describir datos que se pueden clasificar en grupos o categorías específicas, dentro de nuestro dataset se encuentran 2 columnas de eeste tipo, por ejemplo `property_type`, que hace alusión al tipo de propiedad,
la función `value_counts` nos muestra las categorias que tiene la variable y la cantidad (si se quiere normalizar se asigna al parámetro, `normalize`= True):

In [ ]:
# Distribución de datos de la variable 'property_type'
df['property_type'].value_counts(normalize=False)

In [ ]:
df['property_type'].value_counts(normalize=True).head(5).plot.pie(autopct='%1.1f%%')

Para esta variable, los casos se distribuyen la mayoria entre casas (47%) y apartamentos (46.3%), ¿que otras variables son categoricas y cómo se distribuyen? 

In [ ]:
# Que categorias y cuantas de cada una exixten en la variable categórica restante?
#Digite el codigo aquí:


In [ ]:
# Diagrama de torta
#Digite el codigo aquí:


Responda pregunta 8

## Correlación entre columnas
La libreria dython nos ayuda a encontrar la correlación entre las columnas de un dataframe de pandas, no importa que sen noméricas o categoricas:

In [ ]:
correlation = nominal.associations(df, figsize=(20,10), mark_columns=True, annot=False)

Entre mas oscuro sea el color menos correlación hay entre dos columnas, por ejemplo 'host_since' se correlaciona muy poco con 'reviews_per_month', y viceversa, sie la correlación se acerca a amarillo indica que hay una alta correlación, y si se acerca a cyan indica que hay una alta correlación pero es negativa.
La variable correlation que nos entrega dython es la matriz de correlación y esta dada como un DataFrame de pandas. 

In [ ]:
correlation['corr']

## Correlación de las variables respecto a la variable precio, se encuentra ordenado de forma descendente la importancia de las variables para la predicción del precio del alquiler de un inmueble en Airbnb Seatle

In [ ]:
correlation['corr'].loc['price (con)',:].abs().sort_values(ascending=False)

Responda preguntas 9 y 10